In [530]:
## import packages
import os
import sys
sys.path.insert(0, "../")
from MVB.nn import GaussianLinear
from MVB.nn import GaussianConv2d
from MVB.nn import GaussianParameter
from MVB.nn.utils import BlundellPenalty, SetMixturePrior
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import math 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [570]:
n = 5000
p = 1

X = torch.randn(size = (n, p))## generate X of (n=1000, p = 10) each element follows N(0,1)
g_X = torch.exp(X) -  (torch.abs(X)) **3 
# g_X = torch.cat((X[: , 0:1]*0,
#                  X[: , 1:2], 
#                  X[: , 2:3]**3, 
#                  torch.exp(X[: , 3:4]),
#                  torch.sin(2 * math.pi * X[: , 4:5]),
#                  torch.exp(torch.abs (X[: , 5:6]))  - 2 * X[: , 5:6]) , 1)

#beta = torch.empty(p, 1).uniform_(3,5)  ## generate coefficient randomly
beta = 0.5
error = torch.randn(size = (n, 1)) ## generate iid error from N(0,1)
#y1 = g_X @ beta
#y1 = X + 0.3 * torch.sin (2 * math.pi * (X + error)) + 0.4* torch.sin (4 * math.pi * (X + error))
#y1 = (y - torch.mean(y))/ torch.std(y)

y = g_X * beta + error

full_dataset = torch.cat((X, y), 1)

full_data_R = full_dataset.numpy()

beta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
test_dataset = torch.cat((X_test, y_test), 1)
train_dataset = torch.cat((X_train, y_train), 1)

In [717]:
import torch
import pandas as pd

X = pd.read_csv('super_X.csv')
X_tensor = torch.tensor(X.values)
X_tensor = X_tensor.to(torch.float32)
p = X.shape[1]
p
y = pd.read_csv('super_y.csv')
y_tensor = torch.tensor(y.values).to(torch.float32)

In [719]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.1, random_state=42)
test_dataset = torch.cat((X_test, y_test), 1)
train_dataset = torch.cat((X_train, y_train), 1)

In [726]:
h = 200

# a neural network with 1 hidden layer that input 10 dimension x and output 1 dimension y hat with relu as activation function
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = GaussianLinear(p, h)
        self.fc2= GaussianLinear(h,1)
        #self.fc3= GaussianLinear(20,h)
        #self.fc4= GaussianLinear(h,1)
        ## eta = log sigma^2, where sigma^2 is the variance for iid noise in regression setting
        ## to apply our framework, estimate sigma^2 is a must since we need complete likelihood information while combine
        ## with the spike slab prior
        self.eta = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
        #out = self.relu(self.fc2(out))
        #out = self.relu(self.fc3(out))
        out = self.fc2(out)
        return out
    
    ## customer function to compute estimated noise variance from eta
    def getSigmaSq(self):
        return torch.exp(self.eta)

In [727]:
torch.save(train_dataset, 'example_train_data.pt')
torch.save(test_dataset, 'example_test_data.pt')

In [728]:
## code for init model
# set prior parameters
pi = 0.5
tau1 = 10 ## slab sd
tau0 = 1 ## spike sd

num_epochs = 100
batch_size = 64
learning_rate = 0.01
scheduler_step_size = 5
scheduler_gamma = 0.2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet().to(device) ## create a model object from class NeuralNet
SetMixturePrior(model, pi, tau1, tau0) ## inject prior to this neural network

In [729]:
## Code for init the training schema
# set SGD parameters

# set loss(model loglikelihood) type, optimizer and scheduler
criterion = nn.MSELoss() # use MSE loss for regression problem
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                           step_size = scheduler_step_size,
                                           gamma = scheduler_gamma)

In [730]:
## insert data into train loader and test loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [731]:
## Code for training the model
total_step = len(train_loader)
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for observation in train_loader:
        x = observation[:,:p].to(device)
        y = observation[:, p].reshape((len(observation), 1)).to(device)
        yhat = model(x)
        
        ## negative loglikelihood for regression + penalty
        loss = criterion(yhat, y)/(2*model.getSigmaSq()) + torch.log(model.getSigmaSq())/2 + BlundellPenalty(model, len(train_dataset))
        #loss = criterion(yhat, y)/(2*1) + BlundellPenalty(model, len(train_dataset))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    ## report train and test mse at end of each epoch
    model.eval()
    with torch.no_grad():
        train_MSEs = []
        for observation in train_loader:
            x = observation[:,:p].to(device)
            y = observation[:, p].reshape((len(observation), 1)).to(device)
            yhat = model(x)
            train_MSEs.append(criterion(yhat, y).item())
            
        trainMSE = sum(train_MSEs)/len(train_MSEs)
        
        
        test_MSEs = []
        for observation in test_loader:
            x = observation[:,:p].to(device)
            y = observation[:, p].reshape((len(observation), 1)).to(device)
            yhat = model(x)
            test_MSEs.append(criterion(yhat, y).item())
            
        testMSE = sum(test_MSEs)/len(test_MSEs)
    message = 'Epoch: {}, Time: {:.4f}, train MSE: {:.4f} test MSE: {:.4f}'.format(epoch, time.time() - start_time, trainMSE, testMSE)
    print(message)
    print(model.getSigmaSq())
    
    scheduler.step()

Epoch: 0, Time: 3.0909, train MSE: 1181.2370 test MSE: 1131.5659
tensor([1.2023], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 1, Time: 6.2469, train MSE: 736.4270 test MSE: 702.4104
tensor([1.2226], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 2, Time: 9.4073, train MSE: 731.9254 test MSE: 700.9716
tensor([1.2442], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 3, Time: 12.5710, train MSE: 611.8353 test MSE: 589.2113
tensor([1.2681], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 4, Time: 15.7326, train MSE: 583.5148 test MSE: 562.6883
tensor([1.2941], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 5, Time: 18.8927, train MSE: 495.9744 test MSE: 478.4254
tensor([1.2998], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 6, Time: 22.0578, train MSE: 481.9907 test MSE: 464.8583
tensor([1.3064], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 7, Time: 25.2242, train MSE: 557.2577 test MSE: 541.7089
tensor([1.3140], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 8, Time: 28.4323, train MS

Epoch: 67, Time: 213.3120, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 68, Time: 216.4069, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 69, Time: 219.4789, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 70, Time: 222.5484, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 71, Time: 225.6139, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 72, Time: 228.6942, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 73, Time: 231.7638, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 74, Time: 234.8731, train MSE: 452.5049 test MSE: 443.7868
tensor([1.3595], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 75, Time:

In [650]:
torch.save(model, './results/model_bl1.pt')

In [715]:
sel_2 = torch.arange(10)
sel_2
train_dataset = torch.cat( (X_train[:, sel_2],y_train), 1)
test_dataset = torch.cat( (X_test[:, sel_2],y_test), 1)
p = train_dataset.shape[1] - 1

In [716]:
h = 50

# a neural network with 1 hidden layer that input 10 dimension x and output 1 dimension y hat with relu as activation function
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = GaussianLinear(p, h)
        self.fc2= GaussianLinear(h,1)
        #self.fc3= GaussianLinear(20,h)
        #self.fc4= GaussianLinear(h,1)
        ## eta = log sigma^2, where sigma^2 is the variance for iid noise in regression setting
        ## to apply our framework, estimate sigma^2 is a must since we need complete likelihood information while combine
        ## with the spike slab prior
        self.eta = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
        #out = self.relu(self.fc2(out))
        #out = self.relu(self.fc3(out))
        out = self.fc2(out)
        return out
    
    ## customer function to compute estimated noise variance from eta
    def getSigmaSq(self):
        return torch.exp(self.eta)



## code for init model
# set prior parameters
pi = 0.5
tau1 = 10 ## slab sd
tau0 = 1 ## spike sd

num_epochs = 20
batch_size = 64
learning_rate = 0.01
scheduler_step_size = 5
scheduler_gamma = 0.2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet().to(device) ## create a model object from class NeuralNet
SetMixturePrior(model, pi, tau1, tau0) ## inject prior to this neural network

## Code for init the training schema
# set SGD parameters

# set loss(model loglikelihood) type, optimizer and scheduler
criterion = nn.MSELoss() # use MSE loss for regression problem
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                           step_size = scheduler_step_size,
                                           gamma = scheduler_gamma)

## insert data into train loader and test loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

## Code for training the model
total_step = len(train_loader)
start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for observation in train_loader:
        x = observation[:,:p].to(device)
        y = observation[:, p].reshape((len(observation), 1)).to(device)
        yhat = model(x)
        
        ## negative loglikelihood for regression + penalty
        loss = criterion(yhat, y)/(2*model.getSigmaSq()) + torch.log(model.getSigmaSq())/2 + BlundellPenalty(model, len(train_dataset))
        #loss = criterion(yhat, y)/(2*1) + BlundellPenalty(model, len(train_dataset))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    ## report train and test mse at end of each epoch
    model.eval()
    with torch.no_grad():
        train_MSEs = []
        for observation in train_loader:
            x = observation[:,:p].to(device)
            y = observation[:, p].reshape((len(observation), 1)).to(device)
            yhat = model(x)
            train_MSEs.append(criterion(yhat, y).item())
            
        trainMSE = sum(train_MSEs)/len(train_MSEs)
        
        
        test_MSEs = []
        for observation in test_loader:
            x = observation[:,:p].to(device)
            y = observation[:, p].reshape((len(observation), 1)).to(device)
            yhat = model(x)
            test_MSEs.append(criterion(yhat, y).item())
            
        testMSE = sum(test_MSEs)/len(test_MSEs)
    message = 'Epoch: {}, Time: {:.4f}, train MSE: {:.4f} test MSE: {:.4f}'.format(epoch, time.time() - start_time, trainMSE, testMSE)
    print(message)
    print(model.getSigmaSq())
    
    scheduler.step()

Epoch: 0, Time: 3.0476, train MSE: 615.3266 test MSE: 611.4920
tensor([4.8119], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 1, Time: 6.1030, train MSE: 588.1627 test MSE: 586.5842
tensor([10.5126], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 2, Time: 9.1767, train MSE: 572.8918 test MSE: 575.6386
tensor([18.0350], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 3, Time: 12.2445, train MSE: 569.8967 test MSE: 573.5735
tensor([27.2773], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 4, Time: 15.3179, train MSE: 578.3098 test MSE: 577.9179
tensor([38.5034], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 5, Time: 18.4389, train MSE: 563.5893 test MSE: 566.6514
tensor([41.1316], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 6, Time: 21.5784, train MSE: 568.2980 test MSE: 573.2172
tensor([44.2213], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 7, Time: 24.7208, train MSE: 563.2903 test MSE: 564.9672
tensor([47.8395], device='cuda:0', grad_fn=<ExpBackward>)
Epoch: 8, Time: 27.8601, tra